In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
from jax.lib import xla_bridge
print("jax backend {}".format(xla_bridge.get_backend().platform))

from SSN_classes import SSN_mid_local
from training import train_model

jax backend cpu


In [2]:
import jax.numpy as np

from parameters import grid_pars, filter_pars, stimuli_pars, sig_pars, ssn_pars, conn_pars_m, conn_pars_s, ssn_layer_pars, conv_pars, training_pars, loss_pars

# check where A2 is defined in Clara's code
ssn_ori_map_loaded = np.load(os.path.join(os.getcwd(), 'ssn_map_uniform_good.npy'))
#Find normalization constant of Gabor filters
ssn_mid=SSN_mid_local(ssn_pars=ssn_pars, grid_pars=grid_pars, conn_pars=conn_pars_m, filter_pars=filter_pars, J_2x2= ssn_layer_pars.J_2x2_m, gE = ssn_layer_pars.gE_m, gI=ssn_layer_pars.gI_m, ori_map = ssn_ori_map_loaded)
ssn_pars.A = ssn_mid.A
ssn_pars.A2 = ssn_mid.A2
####################### TRAINING PARAMETERS #############################
#Collect constant parameters into single class
class constant_pars:
    ssn_pars = ssn_pars
    s_2x2 = ssn_layer_pars.s_2x2_s
    sigma_oris = ssn_layer_pars.sigma_oris
    grid_pars = grid_pars
    conn_pars_m = conn_pars_m
    conn_pars_s = conn_pars_s
    gE = ssn_layer_pars.gE
    gI = ssn_layer_pars.gI
    filter_pars = filter_pars
    noise_type = 'poisson'
    ssn_ori_map = ssn_ori_map_loaded
    ref_ori = stimuli_pars.ref_ori
    
readout_pars = dict(w_sig = sig_pars.w_sig, b_sig = sig_pars.b_sig)
ssn_layer_pars = dict(J_2x2_m = ssn_layer_pars.J_2x2_m, J_2x2_s = ssn_layer_pars.J_2x2_s, c_E = ssn_layer_pars.c_E, c_I = ssn_layer_pars.c_I, f_E = ssn_layer_pars.f_E, f_I = ssn_layer_pars.f_I, kappa_pre = ssn_layer_pars.kappa_pre, kappa_post = ssn_layer_pars.kappa_post)

########### TRAINING ############
from save_code import save_code
results_dir = save_code()
file_name = 'results.csv'
results_filename = os.path.join(results_dir,file_name)
(
        [ssn_layer_pars, readout_pars],
        val_loss_per_epoch,
        all_losses,
        train_accs,
        train_sig_input,
        train_sig_output,
        val_sig_input,
        val_sig_output,
        epochs_plot,
        save_w_sigs,
    ) = train_model(
    ssn_layer_pars,
    readout_pars,
    constant_pars,
    conv_pars,
    loss_pars,
    training_pars,
    stimuli_pars,
    results_filename
)


Files copied successfully to: results/Nov23_v0
Files copied successfully to: results/Nov23_v1
Training model for 20 epochs  with learning rate 0.001, sig_noise 2.0 at offset 4.0, lam_w 1, batch size 50, noise_type poisson
Saving results to csv  results/Nov23_v1\results.csv
Training loss: 1.6037598848342896 ¦ Validation -- loss: 1.3124967813491821, true accuracy: 0.5799999833106995, at epoch 1, (time 53.552123069763184, 0.4602620601654053), 
Training loss: 1.462250828742981 ¦ Validation -- loss: 1.6904652118682861, true accuracy: 0.46000000834465027, at epoch 6, (time 1.8416569232940674, 0.8827865123748779), 
Training loss: 1.2602494955062866 ¦ Validation -- loss: 1.4675003290176392, true accuracy: 0.5400000214576721, at epoch 13, (time 1.189713478088379, 0.5517909526824951), 
Training loss: 1.4688515663146973 ¦ Validation -- loss: 1.4429302215576172, true accuracy: 0.5, at epoch 20, (time 1.429305076599121, 0.5559267997741699), 
Entering second stage at epoch 20
Training loss: 1.264247

In [3]:
print(ssn_layer_pars['J_2x2_m'])

[[0.64614874 0.0206267 ]
 [1.3055052  0.51773465]]
